# Imports

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sksurv.datasets import load_whas500, load_gbsg2, load_aids
import data_processing
import plots
import models

# Generate synthetic data

In [ ]:
orig_data = data_processing.load_prepare_aids_dataset()

In [ ]:
_, df_syn = data_processing.generate_synthetic_data(orig_data, 40000, censoring_strategy='covariate_dependent')

In [ ]:
time, censor = data_processing.determine_censor_time(df_syn)
# we can generate extra data and use only the censored samples from a given point to get a dataset with higher censoring percentage 
df_tmp = df_syn.iloc[20001:40000, :]
df_tmp2 = df_syn.iloc[0:20001, :]
df_tmp2 = pd.concat([df_tmp2, df_tmp.loc[(df_tmp['time'] < time) & (df_tmp['censor'] == False)]])

In [ ]:
df_syn = df_tmp2
df_syn.to_excel('datasets/syn/sksurv_aids_40k_covariate.xlsx')

# Experiment
## Load pre-saved data

In [ ]:
df_syn = pd.read_excel("datasets/syn/sksurv_whas500.xlsx", index_col=0)

In [ ]:
# plot censored percentage by time if we want to manually pick the timeframe, can also use data_processing.determine_censor_time
plots.censored_by_time_plot(df_syn)
# params dict for hyperparameter tuning
params = {'n_estimators': [100], 'max_depth': [4, 8, 16, None], 'min_samples_split': [2, 5, 10], 'max_features': ['sqrt', 'log2']}

## Hyperparameter tuning

In [ ]:
# set censor time manually
censor_time = 1500

In [ ]:
rsf_params, best_auc_rsf = models.model_tuning_rsf(df_syn, censor_time, params, 3)

test size: 6223
Iteration 1/24, total time: 78s, 78s/iter, score: 0.9062418802522899, params: {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 2, 'max_features': 'sqrt'}
Iteration 2/24, total time: 139s, 69s/iter, score: 0.9064718056045612, params: {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 2, 'max_features': 'log2'}
Iteration 3/24, total time: 210s, 70s/iter, score: 0.906104712509858, params: {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 5, 'max_features': 'sqrt'}
Iteration 4/24, total time: 282s, 70s/iter, score: 0.9077814885255863, params: {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 5, 'max_features': 'log2'}
Iteration 5/24, total time: 421s, 84s/iter, score: 0.9160123217791916, params: {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 10, 'max_features': 'sqrt'}
Iteration 6/24, total time: 527s, 87s/iter, score: 0.9119746888914488, params: {'n_estimators': 100, 'max_depth': 4, 'min_samples_split': 10, 'max_features': 'l

In [ ]:
datasets = data_processing.create_datasets(df_syn, censor_time)
rfc_params, best_auc_rfc = models.model_tuning(datasets['uncensored']['X_train'],datasets['uncensored']['y_train'], RandomForestClassifier(), params, verbose=3)

In [ ]:
print(f"rsf params: {rsf_params}")
print(f"rfc params: {rfc_params}")

## Train and score models

In [ ]:
# if we want to manually change params
rsf_params = {'n_estimators': 100, 'max_depth': 8, 'min_samples_split': 5, 'max_features': 'sqrt'}
rfc_params = {'max_depth': 64, 'min_samples_leaf': 5, 'n_estimators': 100}

In [ ]:
# if we want to use default params
rsf_params = None
rfc_params = None

In [ ]:
import experiments
rfc_auc_mean = 0
_, top_percent = data_processing.determine_censor_time(df_syn)
censor_vals = [x / 100.0 for x in range(0, int(top_percent), 10)]
print(f'censored ratios: {censor_vals}')
rsf_auc_mean = np.zeros(len(censor_vals))
num_experiments = 1
# censor_time, _ = data_processing.determine_censor_time(df_syn)
for i in tqdm(range(num_experiments)):
    rfc_auc, rsf_aucs = experiments.rsf_rfc(df_syn, censor_vals, censor_time, rsf_params, rfc_params)
    rfc_auc_mean += rfc_auc
    rsf_auc_mean += np.asarray(rsf_aucs)
rfc_auc_mean /= num_experiments
rsf_auc_mean /= num_experiments

In [ ]:
plots.line_plot(rsf_auc_mean, rfc_auc_mean, np.asarray(censor_vals)*100, num_experiments)

In [ ]:
experiments.exp_report(df_syn, 'breast_cancer_md64', censor_vals, censor_time, rsf_params, rfc_params, {'rfc_auc_mean': rfc_auc_mean, 'rsf_auc_mean': rsf_auc_mean})